In [ ]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv('data_adaboost.csv', header=None)

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeClassifier
import math

class AdaboostClassifierCustom(BaseEstimator):
    def __init__(self, n_estimators=3, max_depth=1, random_state=17):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.random_state = random_state
        self.stumps = []
        self.sample_weight = []
        
    def fit(self, X, y):

        self.sample_weight = np.ones(X.shape[0]) / X.shape[0]

        for i in range(self.n_estimators):
          epsilot_t = 1
          while epsilot_t > 0.5:
            stump = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state)
            stump.fit(X, y, sample_weight = self.sample_weight)
            pred = stump.predict(X)
            index = np.where(pred != y)[0]
            epsilot_t = self.sample_weight[index].sum()
            alpha_t = math.log((1 - epsilot_t) / epsilot_t) / 2
            
          self.sample_weight = self.sample_weight * np.exp(-alpha_t * y * pred)
          self.sample_weight = self.sample_weight / self.sample_weight.sum()
          self.stumps.append(stump)
        return self

    def predict(self, X):
        predictions = self.stumps[0].predict(X)
        for i in range(1, self.n_estimators):
            predictions = np.vstack((predictions, self.stumps[i].predict(X)))
        return np.sign(predictions.sum(axis=0))

In [ ]:
X = data.values[:,:2]
y = data.values[:,2]

In [ ]:
stumps = AdaboostClassifierCustom()
stumps.fit(X, y)

pred = stumps.predict(X)

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y, pred)

1.0

In [ ]:
cond = [stumps.stumps[i].tree_.threshold for i in range(stumps.n_estimators)]
cond

[array([ 3.20000005, -2.        , -2.        ]),
 array([ 3., -2., -2.]),
 array([ 1.64999998, -2.        , -2.        ])]